ELASTIC STRUCTURE ANALYSIS OF 2-D STRUCTURES

In [ ]:
import pandas as pd
import numpy as np
import math
from tabulate import tabulate

In [ ]:
member_info=pd.read_csv("/content/MemberData_Additional.csv",encoding="windows-1252")

In [ ]:
nodal_info=pd.read_csv("/content/Nodal_Data_Additional.csv",encoding="windows-1252")

In [ ]:
member_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   S. No.    2 non-null      int64  
 1   N1        2 non-null      int64  
 2   N2        2 non-null      int64  
 3   L(m)      2 non-null      int64  
 4   B(m)      2 non-null      float64
 5   D(m)      2 non-null      float64
 6   E(KN/m2)  2 non-null      int64  
 7   w1(KN/m)  2 non-null      int64  
 8   w2(KN/m)  2 non-null      int64  
 9   P(KN)     2 non-null      int64  
 10  a(m)      2 non-null      int64  
 11  b(m)      2 non-null      int64  
dtypes: float64(2), int64(10)
memory usage: 320.0 bytes


In [ ]:
member_info.head()

,S. No.,N1,N2,L(m),B(m),D(m),E(KN/m2),w1(KN/m),w2(KN/m),P(KN),a(m),b(m)
0,1,1,2,5,0.2,0.2,200000000,0,0,0,0,0
1,2,2,3,5,0.2,0.2,200000000,0,0,0,0,0


In [ ]:
type(member_info['N1'][0])

numpy.int64

In [ ]:
len(member_info.index)

2

In [ ]:
nodal_info.head()

,S. No.,X Coord(m),Y Coord(m),Nodal Load (Fx) (KN),Nodal Load (Fy) (KN),Nodal Load (Mz) (KN-m),Ux,Vy,Theta
0,1,2,4,0,0,0,NaN,0.005,0
1,2,5,8,0,-100,0,1.0,1.000,1
2,3,8,4,0,0,0,0.0,0.000,0


In [ ]:
nodal_info.iloc[1,3]

0

Defining basic functions

In [ ]:
def Ffl (P,a,b,w1,w2,l):  #this function generates fixed end forces matrix of a member for a given udl
  output = [0,0,0,0,0,0]  #[Hn,Vn,Mn,Hn+1,Vn+1,Mn+1]

  if(P!=0):
    m_p1=P*b*b*a/(l*l)
    m_p2=-P*a*a*b/(l*l)
    v_p2=(m_p2-m_p1+P*a)/l
    v_p1=P-v_p2
    output[1]+=v_p1
    output[2]+=m_p1
    output[4]+=v_p2
    output[5]+=m_p2

  if(w1!= 0 or w2 != 0):
    if (w1 == w2):
      v_= w2*l/2
      m_ = w2*l*l/12
      v_1=v_
      v_2=v_
      m_1 = m_
      m_2 = -m_
    elif (w1>w2):
      v_= w2*l/2
      m_ = w2*l*l/12
      v_1 = 7*(w1-w2)*l/20 + v_
      v_2 = 3*(w1-w2)*l/20 + v_
      m_1 = (w1-w2)*l*l/20 + m_
      m_2 = -(w1-w2)*l*l/30 - m_
    elif (w2>w1):
      v_= w1*l/2
      m_ = w1*l*l/12
      v_2 = 7*(w2-w1)*l/20 + v_
      v_1 = 3*(w2-w1)*l/20 + v_
      m_2 = -(w2-w1)*l*l/20 - m_
      m_1 = (w2-w1)*l*l/30 + m_
    output[1]+=v_1
    output[2]+=m_1
    output[4]+=v_2
    output[5]+=m_2

  return(output)

In [ ]:
def Fl(h,v,m):  #n represents member node numbers
  output = [0,0,0]
  k=0
  for i in range(3):
   output[0]=h
   output[1]=v
   output[2]=m
  return(output)

In [ ]:
def dl(n,m):  #n represents member node numbers
  output = [0,0,0,0,0,0]
  k=0
  #print(n,m)
  for i in [n,m]:
    for j in range(3):
      #print(i-1,j+3)
      output[k]= nodal_info.iloc[i-1,j+6]
      #print(output[k])
      k+=1
  return(output)

In [ ]:
def kl (A,E,L,I): #this function generates local stifness matrix for a member
  output = np.zeros((6,6))
  a=A*E/L
  b=12*E*I/(L*L*L)
  c=6*E*I/(L*L)
  d=4*E*I/L
  e=2*E*I/L
  output[0,0]=a
  output[0,3]=-a
  output[1,1]=b
  output[1,2]=c
  output[1,4]=-b
  output[1,5]=c
  output[2,1]=c
  output[2,2]=d
  output[2,4]=-c
  output[2,5]=e
  output[3,0]=-a
  output[3,3]=a
  output[4,1]=-b
  output[4,2]=-c
  output[4,4]=b
  output[4,5]=-c
  output[5,1]=c
  output[5,2]=e
  output[5,4]=-c
  output[5,5]=d
  return (output)

In [ ]:
kl(1,1,1,1)

array([[  1.,   0.,   0.,  -1.,   0.,   0.],
       [  0.,  12.,   6.,   0., -12.,   6.],
       [  0.,   6.,   4.,   0.,  -6.,   2.],
       [ -1.,   0.,   0.,   1.,   0.,   0.],
       [  0., -12.,  -6.,   0.,  12.,  -6.],
       [  0.,   6.,   2.,   0.,  -6.,   4.]])

In [ ]:
def TM(z):  #z represents ϴ, TM represents transformation matrix
            #provide angle in radians
            #this function generates Transformation matrix for a given member
  a=math.cos(z)
  b=math.sin(z)
  output = np.zeros((6,6))
  output[0,0]=a
  output[0,1]=-b
  output[1,0]=b
  output[1,1]=a
  output[2,2]=1
  output[3,3]=a
  output[3,4]=-b
  output[4,3]=b
  output[4,4]=a
  output[5,5]=1

  return(output)

In [ ]:
TM(0)

array([[ 1., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1., -0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.]])

In [ ]:
def GSM (T,k): # T transform matrix, k Local Stifness matrix
               #this function converts local stifness matrix to global stifness matrix for a given member.
  t= np.array(T)
  k=np.array(k)
  test = t.dot(k)
  z=np.transpose(t)
  out=test.dot(z)
  return(out)

In [ ]:
def Gffl (T,Ffl):

  t= np.array(T)
  Fl=np.array(Ffl)
  out = t.dot(Fl)
  return (out)

Main functions leading to assembly

In [ ]:
Kl=[]       #Kl contains all the local stifness matrix of all members
for i in range (len(member_info.index)):
  A = (member_info['B(m)'][i])*(member_info['D(m)'][i])
  E = member_info['E(KN/m2)'][i]
  L = member_info['L(m)'][i]
  I = ((member_info['B(m)'][i])*((member_info['D(m)'][i])**3))/12
  k=kl(A,E,L,I)
  Kl.append(k)

In [ ]:
FFl=[]      #FFl contains all the local fixed end forces matrix of all members
for i in range (len(member_info.index)):
  P = member_info['P(KN)'][i]
  a = member_info['a(m)'][i]
  b = member_info['b(m)'][i]
  w1 = member_info['w1(KN/m)'][i]
  w2 = member_info['w2(KN/m)'][i]
  l = member_info['L(m)'][i]
  f=Ffl(P,a,b,w1,w2,l)
  FFl.append(f)

In [ ]:
z=np.zeros((len(member_info.index)))  #z contains angle of each members
print(z)
for i in range(len(member_info.index)):
  # print(i)
  n1=member_info['N1'][i]
  n2=member_info['N2'][i]
  x1=nodal_info['X Coord(m)'][n1-1]
  y1=nodal_info['Y Coord(m)'][n1-1]
  x2=nodal_info['X Coord(m)'][n2-1]
  y2=nodal_info['Y Coord(m)'][n2-1]
  # print(n1,n2)
  # print(x1,y2,x2,y2)
  if (x1 == x2):
    if(y2>y1):
      angle=((22/7)/2)
    if(y2<y1):
      angle=(-1*(22/7)/2)
  else:
    test=(y2-y1)/(x2-x1)
    angle=np.arctan(test)
  z[i] += angle
print(z)

[0. 0.]
[ 0.92729522 -0.92729522]


In [ ]:
FFg=[]        #FFg and Kg contains global Fixed end forces and global stiffness
Kg=[]           #matrices for all members
T=[]               # T contains Transformation matrix for all members
for i in range(len(member_info.index)):

  t=TM(z[i])
  T.append(t)

  kg=GSM(t,Kl[i])
  Kg.append(kg)

  ffg=Gffl(t,FFl[i])
  FFg.append(ffg)

In [ ]:
print(FFg)

[array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]


In [ ]:
#Making assembly for kg === Kga      #Kga is global stifness matrix for the assembly
#Making assembly for FFg === Ffga     #FFga is global Fixed end force matrix for the assembly
Kga=np.zeros((3*(len(nodal_info.index)),3*(len(nodal_info.index))))
Ffga=np.zeros((3*(len(nodal_info.index)),1))
for i in range(len(member_info.index)):
  n1=member_info['N1'][i]
  n2=member_info['N2'][i]
  list= [3*n1-2,3*n1-1,3*n1,3*n2-2,3*n2-1,3*n2]
  for j in range(6):
    Ffga[list[j]-1][0]+=FFg[i][j]
    for k in range(6):
      Kga[list[j]-1][list[k]-1]+= Kg[i][j][k]
      # if(list[j]==22 and list[k]==22):
      #   print(i,j,k)

In [ ]:
print(Ffga)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [ ]:
#Making assembly for Fg === Fga      #Fga is global force matrix for the assembly
Fga=np.zeros((3*(len(nodal_info.index)),1))
for i in range(1,len(nodal_info.index)+1):
  Fga[3*i-2-1][0] =nodal_info['Nodal Load (Fx) (KN)'][i-1]
  Fga[3*i-1-1][0]=nodal_info['Nodal Load (Fy) (KN)'][i-1]
  Fga[3*i-1][0]=nodal_info['Nodal Load (Mz) (KN-m)'][i-1]

In [ ]:
print(Fga)

[[   0.]
 [   0.]
 [   0.]
 [   0.]
 [-100.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]]


In [ ]:
Active_degrees=[]
Restrained_degrees=[]
Support_nodes=[]
Internal_nodes=[]
# Fga_aa=[]

for i in range(len(nodal_info.index)):

  if((nodal_info['Ux'][i])!=1 or (nodal_info['Vy'][i])!=1 or (nodal_info['Theta'][i])!=1):
    Support_nodes.append(i)
  else:
    Internal_nodes.append(i)

  if((nodal_info['Ux'][i])==1):
    Active_degrees.append(3*(i+1)-2-1)
    # Fga_aa.append()
  else:
     Restrained_degrees.append(3*(i+1)-2-1)

  if((nodal_info['Vy'][i])==1):
    Active_degrees.append(3*(i+1)-1-1)
  else:
     Restrained_degrees.append(3*(i+1)-1-1)

  if((nodal_info['Theta'][i])==1):
    Active_degrees.append(3*(i+1)-1)
  else:
     Restrained_degrees.append(3*(i+1)-1)

In [ ]:
print(Active_degrees)
print(Restrained_degrees)
print(Support_nodes)
print(Internal_nodes)

[3, 4, 5]
[0, 1, 2, 6, 7, 8]
[0, 2]
[1]


In [ ]:
kga_aa= np.zeros((len(Active_degrees),len(Active_degrees)))
for i in range(len(Active_degrees)):
  for j in range(len(Active_degrees)):
    kga_aa[i][j]+=Kga[Active_degrees[i]][Active_degrees[j]]

kga_ar= np.zeros((len(Active_degrees),len(Restrained_degrees)))
for i in range(len(Active_degrees)):
  for j in range(len(Restrained_degrees)):
    kga_ar[i][j]+=Kga[Active_degrees[i]][Restrained_degrees[j]]

Fga_aa= np.zeros((len(Active_degrees)))
for i in range (len(Active_degrees)):
  Fga_aa[i]+=Fga[Active_degrees[i]][0]

Ffga_aa = np.zeros((len(Active_degrees)))
for i in range (len(Active_degrees)):
  Ffga_aa[i]+=Ffga[Active_degrees[i]][0]

F_aa= np.zeros((len(Active_degrees)))
for i in range (len(Active_degrees)):
  F_aa[i]=Fga_aa[i]-Ffga_aa[i]

In [ ]:
# print(Ffga)
# print(Ffga_aa)
print(F_aa)

[   0. -100.    0.]


In [ ]:
dga= np.zeros((len(Active_degrees))+(len(Restrained_degrees)))  #loop for computing active
dga_aa = np.zeros((len(Active_degrees)))                        #degrees of freedom
dga_rr= np.zeros((len(Restrained_degrees)))

for j in range(len(Restrained_degrees)):
  if ((Restrained_degrees[j])%3 ==2):
    dga_rr[j] += nodal_info['Theta'][int((Restrained_degrees[j])/3)]
    dga[Restrained_degrees[j]]+= nodal_info['Theta'][int((Restrained_degrees[j])/3)]
    print(Restrained_degrees[j],dga[Restrained_degrees[j]])
  elif ((Restrained_degrees[j])%3 ==1):
    dga_rr[j] += nodal_info['Vy'][int((Restrained_degrees[j])/3)]
    dga[Restrained_degrees[j]]+= nodal_info['Vy'][int((Restrained_degrees[j])/3)]
    print(Restrained_degrees[j],dga[Restrained_degrees[j]])
  elif ((Restrained_degrees[j])%3 ==1):
   dga_rr[j] +=  nodal_info['Ux'][int((Restrained_degrees[j])/3)]
   dga[Restrained_degrees[j]]+= nodal_info['Ux'][int((Restrained_degrees[j])/3)]
   print(Restrained_degrees[j],dga[Restrained_degrees[j]])

# test1=kga_ar.dot(dga_ar)
dummy = kga_ar.dot(dga_rr)
# print(dummy)
dummy2 = []
for i in range(len(F_aa)):
  dummy3= F_aa[i]-dummy[i]
  dummy2.append(dummy3)
test_da=(np.linalg.inv(kga_aa)).dot(np.array(dummy2))

# # test_da=(np.linalg.inv(kga_aa)).dot(F_aa)
# # print(test_da)
# # print(type(test_da))
test_dar = np.round(test_da,decimals=2)
# print(test_dar)
for i in range(len(test_da)):
  dga_aa[i] += test_da[i]
  # dga_aa[i]+=test_dar[i]
# print(F_aa)
# print(kga_aa)
for i in range (len(Active_degrees)):
  dga[Active_degrees[i]]+=dga_aa[i]

1 0.005
2 0.0
7 0.0
8 0.0


In [ ]:
print(dga_rr)

[0.    0.005 0.    0.    0.    0.   ]


In [ ]:
print(np.round(dga,decimals=2))

[ 0.  0.  0.  0.  0. -0.  0.  0.  0.]


In [ ]:
testfa=[]
test= Kga.dot(dga)
print(np.round(test,decimals=2))
for i in range (len(dga)):
  test1 = test[i]+Ffga[i][0]
  testfa.append(test1)


[  37.41   54.44   13.51    0.   -100.      0.    -37.41   45.56   13.14]


In [ ]:
Final_Fga=np.round_(testfa,decimals=2)    #Final_Fga and Final_Dga represents the global
Final_Dga=np.round_(dga,decimals=5)       #force vector and gloval displacement vector of assembly

In [ ]:
print(Final_Fga)

[  37.41   54.44   13.51    0.   -100.      0.    -37.41   45.56   13.14]


In [ ]:
print(Final_Dga)

[ 0.       0.005    0.       0.00333  0.00245 -0.00125  0.       0.
  0.     ]


In [ ]:
M_dg=[]
for i in range(len(member_info.index)):
  d=np.zeros((6))
  n1=member_info['N1'][i]
  n2=member_info['N2'][i]

  d[0]+=dga[3*n1-2-1]
  d[1]+=dga[3*n1-1-1]
  d[2]+=dga[3*n1-1]
  d[3]+=dga[3*n2-2-1]
  d[4]+=dga[3*n2-1-1]
  d[5]+=dga[3*n2-1]

  M_dg.append(d)

In [ ]:
Local_Displacement=[]
for i in range(len(member_info.index)):
  t=TM(z[i])
  dummy_d = np.linalg.inv(t)
  dummy_d1=dummy_d.dot(M_dg[i])
  Local_Displacement.append(dummy_d1)

In [ ]:
MemberEndForces=[]
for i in range(len(member_info.index)):
  f=np.zeros((6))
  mid_var=Kl[i].dot(Local_Displacement[i])
  for j in range(6):
    f[j]+=mid_var[j]+FFl[i][j]
  MemberEndForces.append(f)
MemberEndForces_app = np.round(MemberEndForces,decimals=2)

In [ ]:
Nodes_KnownDoF=[]
Nodes_UnknownDoF=[]

for i in range(len(Active_degrees)):
  n=int(Active_degrees[i]/3)
  if(n not in Nodes_UnknownDoF):
    Nodes_UnknownDoF.append(n)

for i in range(len(Restrained_degrees)):
  n=int(Restrained_degrees[i]/3)
  if(n not in Nodes_KnownDoF):
    Nodes_KnownDoF.append(n)

Post_Processing Results

In [ ]:
print("DISPLACEMENTS")

print()

print("Known Degrees of Freedom")
Data1=[]
for i in range(len(Nodes_KnownDoF)):
  data1=[]
  print("Node ",Nodes_KnownDoF[i]+1)
  data1.append(Nodes_KnownDoF[i]+1)
  if((3*(Nodes_KnownDoF[i]+1)-2-1) in Restrained_degrees):
    print("Ux -> ",1000*Final_Dga[3*(Nodes_KnownDoF[i]+1)-2-1],"mm" ,end=' ')
    data1.append(1000*Final_Dga[3*(Nodes_KnownDoF[i]+1)-2-1])
  if((3*(Nodes_KnownDoF[i]+1)-1-1) in Restrained_degrees):
    print("Vy -> ",1000*Final_Dga[3*(Nodes_KnownDoF[i]+1)-1-1],"mm" ,end=' ')
    data1.append(1000*Final_Dga[3*(Nodes_KnownDoF[i]+1)-1-1])
  if((3*(Nodes_KnownDoF[i]+1)-1) in Restrained_degrees):
    print("ϴz -> ",Final_Dga[3*(Nodes_KnownDoF[i]+1)-1],end=' ')
    data1.append(Final_Dga[3*(Nodes_KnownDoF[i]+1)-1])
  Data1.append(data1)
  print()
print()
print()

print("UnKnown Degrees of Freedom")
Data2=[]
for i in range(len(Nodes_UnknownDoF)):
  data2=[]
  print("Node ",Nodes_UnknownDoF[i]+1)
  data2.append(Nodes_UnknownDoF[i]+1)
  if((3*(Nodes_UnknownDoF[i]+1)-2-1) in Active_degrees):
    print("Ux -> ",1000*Final_Dga[3*(Nodes_UnknownDoF[i]+1)-2-1],"mm" ,end=' ')
    data2.append(1000*Final_Dga[3*(Nodes_UnknownDoF[i]+1)-2-1])
  if((3*(Nodes_UnknownDoF[i]+1)-1-1) in Active_degrees):
    print("Vy -> ",1000*Final_Dga[3*(Nodes_UnknownDoF[i]+1)-1-1],"mm" ,end=' ')
    data2.append(1000*Final_Dga[3*(Nodes_UnknownDoF[i]+1)-1-1])
  if((3*(Nodes_UnknownDoF[i]+1)-1) in Active_degrees):
    print("ϴz -> ",Final_Dga[3*(Nodes_UnknownDoF[i]+1)-1],end=' ')
    data2.append(Final_Dga[3*(Nodes_UnknownDoF[i]+1)-1])
  Data2.append(data2)
  print()
print()
print()


DISPLACEMENTS

Known Degrees of Freedom
Node  1
Ux ->  0.0 mm Vy ->  5.0 mm ϴz ->  0.0 
Node  3
Ux ->  0.0 mm Vy ->  0.0 mm ϴz ->  0.0 


UnKnown Degrees of Freedom
Node  2
Ux ->  3.33 mm Vy ->  2.4499999999999997 mm ϴz ->  -0.00125 




In [ ]:
# # from tabulate import tabulate
# print("Displacements")
# print()
# print("Known Degrees of Freedom")
# print (tabulate(Data1, headers=["Node", "Ux(mm)", "Vy(mm)", "θz"]))
# print()
# print("UnKnown Degrees of Freedom")
# print (tabulate(Data2, headers=["Node", "Ux(mm)", "Vy(mm)", "θz"]))

In [ ]:
print("Support Reaction")
Data=[]
for i in range (len(Support_nodes)):
  data=[]
  print("Node ",Support_nodes[i]+1)
  data.append(Support_nodes[i]+1)
  if((3*(Support_nodes[i]+1)-2-1) in Restrained_degrees):
    print("Hx -> ",Final_Fga[3*(Support_nodes[i]+1)-2-1]," KN",end=' ')
    data.append(Final_Fga[3*(Support_nodes[i]+1)-2-1])
  if((3*(Support_nodes[i]+1)-1-1) in Restrained_degrees):
    print("Vy -> ",Final_Fga[3*(Support_nodes[i]+1)-1-1]," KN",end=' ')
    data.append(Final_Fga[3*(Support_nodes[i]+1)-1-1])
  if((3*(Support_nodes[i]+1)-1) in Restrained_degrees):
    print("Mz -> ",Final_Fga[3*(Support_nodes[i]+1)-1]," KN-m",end=' ')
    data.append(Final_Fga[3*(Support_nodes[i]+1)-1])
  Data.append(data)
  print()
  print()

Support Reaction
Node  1
Hx ->  37.41  KN Vy ->  54.44  KN Mz ->  13.51  KN-m 

Node  3
Hx ->  -37.41  KN Vy ->  45.56  KN Mz ->  13.14  KN-m 



In [ ]:
# # from tabulate import tabulate
# print("Support Reaction")
# print()
# print (tabulate(Data, headers=["Node", "Hx(KN)", "Vy(KN)", "Mz(KN-m)"]))

In [ ]:
print("Member End Forces")
print()
Data3=[]
for i in range (len(member_info.index)):
  data3=[]
  n1=member_info['N1'][i]
  n2=member_info['N2'][i]
  print("Member ",i+1)
  data3.append(i+1)
  print("U",n1," -> ",MemberEndForces[i][0]," KN",end='   ')
  data3.append(MemberEndForces_app[i][0])
  print("V",n1," -> ",MemberEndForces[i][1]," KN",end='   ')
  data3.append(MemberEndForces_app[i][1])
  print("M",n1," -> ",MemberEndForces[i][2]," KN-m",end='   ')
  data3.append(MemberEndForces_app[i][2])
  print()
  print("U",n2," -> ",MemberEndForces[i][3]," KN",end='   ')
  data3.append(MemberEndForces_app[i][3])
  print("V",n2," -> ",MemberEndForces[i][4]," KN",end='   ')
  data3.append(MemberEndForces_app[i][4])
  print("M",n2," -> ",MemberEndForces[i][5]," KN-m",end=' ')
  data3.append(MemberEndForces_app[i][5])
  Data3.append(data3)
  print()
  print()

Member End Forces

Member  1
U 1  ->  65.99682953666434  KN   V 1  ->  2.7397042785848775  KN   M 1  ->  13.511189991185942  KN-m   
U 2  ->  -65.99682953666434  KN   V 2  ->  -2.7397042785848775  KN   M 2  ->  0.18733140173843665  KN-m 

Member  2
U 2  ->  58.89077162229349  KN   V 2  ->  2.5898391571941284  KN   M 2  ->  -0.18733140173843665  KN-m   
U 3  ->  -58.89077162229349  KN   V 3  ->  -2.5898391571941284  KN   M 3  ->  13.136527187709069  KN-m 



In [ ]:
from tabulate import tabulate
print("Member End Forces")
print()
print("Known Degrees of Freedom")
print (tabulate(Data3, headers=["Member", "U1(KN)", "V1(KN)", "M1(KN-m)", "U2(KN)", "V2(mm)", "M2(KN-m)"]))

Member End Forces

Known Degrees of Freedom
  Member    U1(KN)    V1(KN)    M1(KN-m)    U2(KN)    V2(mm)    M2(KN-m)
--------  --------  --------  ----------  --------  --------  ----------
       1     66         2.74       13.51    -66        -2.74        0.19
       2     58.89      2.59       -0.19    -58.89     -2.59       13.14
